# TensorFlow 2.0

In [1]:
import os
from glob import glob
from datetime import datetime

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets 

import matplotlib.pyplot as plt

%load_ext tensorboard
%matplotlib inline

## Hyperparameter Tunning

In [2]:
num_epochs = 5
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.5

input_shape = (32, 32, 3)
num_classes = 10

## Build Model

In [3]:
inputs = layers.Input(input_shape)

net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [4]:
# Model is the full model w/o custom layers
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
              loss='sparse_categorical_crossentropy',  # Loss Function 
              metrics=['accuracy'])  # Metrics / Accuracy

# Data Preprocess

In [5]:
train_paths = glob('dataset/cifar/train/*.png')[:100]
test_paths = glob('dataset/cifar/test/*.png')[:100]

In [6]:
def get_class_name(path):
    return path.split('_')[-1].replace('.png', '')

In [7]:
train_labels = [get_class_name(path) for path in train_paths]
class_names = np.unique(train_labels)

In [8]:
def get_label(path):
    fname = tf.strings.split(path, '_')[-1]
    lbl_name = tf.strings.regex_replace(fname, '.png', '')
    onehot = tf.cast(lbl_name == class_names, tf.uint8)
    return tf.argmax(onehot)  # 이번에는 onehot이 아닌 label 번호로

In [9]:
def load_image_label(path):
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile)
    image = tf.cast(image, tf.float32) / 255.  # rescale
    
    label = get_label(path)
    return image, label

In [10]:
def image_preprocess(image, label):
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_flip_left_right(image)
    return image, label

In [11]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [12]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
train_dataset = train_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.map(image_preprocess, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(buffer_size=len(train_paths))
train_dataset = train_dataset.repeat()

In [13]:
test_dataset = tf.data.Dataset.from_tensor_slices(test_paths)
test_dataset = test_dataset.map(load_image_label, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.repeat()

## Training

http://localhost:6006

In [14]:
steps_per_epoch = len(train_paths) // batch_size
validation_steps = len(test_paths) // batch_size

history = model.fit_generator(
    train_dataset,
    steps_per_epoch=steps_per_epoch,
    validation_data=test_dataset,
    validation_steps=validation_steps,
    epochs=num_epochs
)

/Users/lecturer/opt/anaconda3/envs/cvtf2/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
3/3 [==============================] - 2s 641ms/step - loss: 2.4060 - accuracy: 0.1146 - val_loss: 2.3131 - val_accuracy: 0.0938
Epoch 2/5
3/3 [==============================] - 0s 163ms/step - loss: 2.3098 - accuracy: 0.1097 - val_loss: 2.3198 - val_accuracy: 0.0729
Epoch 3/5
3/3 [==============================] - 0s 118ms/step - loss: 2.3550 - accuracy: 0.1018 - val_loss: 2.3190 - val_accuracy: 0.0729
Epoch 4/5
3/3 [==============================] - 0s 125ms/step - loss: 2.2782 - accuracy: 0.1628 - val_loss: 2.3181 - val_accuracy: 0.0729
Epoch 5/5
3/3 [==============================] - 0s 104ms/step - loss: 2.2826 - accuracy: 0.1469 - val_loss: 2.3224 - val_accuracy: 0.0729


# Load Checkpoint

In [ ]:
model.load_weights('checkpoints')

# Saving Model

In [16]:
save_path = 'my_model.h5'

In [17]:
model.save(save_path, include_optimizer=True)

In [18]:
service = tf.keras.models.load_model('my_model.h5')

In [20]:
path = train_paths[0]
test_image, test_label = load_image_label(path)
test_image = test_image[tf.newaxis, ...]

In [21]:
pred = service.predict(test_image)

In [22]:
np.argmax(pred)

2

# Saving Model - 2

In [23]:
model.save_weights('my_model_weight.h5')

In [24]:
with open('model_architercture.json', 'w') as f:
    f.write(model.to_json())

In [25]:
from tensorflow.keras.models import model_from_json

In [28]:
with open('model_architercture.json', 'r') as f:
    model_json = model_from_json(f.read())

In [30]:
model_json.load_weights('my_model_weight.h5')

In [31]:
pred = model_json.predict(test_image)
np.argmax(pred)

2

# model.h5 들여다보기

In [33]:
!conda list h5py

# packages in environment at /Users/lecturer/opt/anaconda3/envs/cvtf2:
#
# Name                    Version                   Build  Channel
h5py                      2.10.0                   pypi_0    pypi


In [32]:
import h5py

In [34]:
model_file = h5py.File('my_model.h5', 'r+')

In [35]:
model_file.keys()

<KeysViewHDF5 ['model_weights', 'optimizer_weights']>

In [36]:
model_file['model_weights'].keys()

<KeysViewHDF5 ['activation', 'activation_1', 'activation_2', 'activation_3', 'activation_4', 'activation_5', 'conv2d', 'conv2d_1', 'conv2d_2', 'conv2d_3', 'dense', 'dense_1', 'dropout', 'dropout_1', 'dropout_2', 'flatten', 'input_1', 'max_pooling2d', 'max_pooling2d_1']>

In [39]:
model_file['model_weights']['conv2d']['conv2d'].keys()

<KeysViewHDF5 ['bias:0', 'kernel:0']>

In [41]:
model_file['model_weights']['conv2d']['conv2d']['kernel:0']

<HDF5 dataset "kernel:0": shape (3, 3, 3, 32), type "<f4">

In [42]:
np.array(model_file['model_weights']['conv2d']['conv2d']['kernel:0'])

array([[[[ 1.19540710e-02,  1.19172558e-01, -3.49400453e-02,
           8.45298618e-02, -8.50088801e-03,  1.21980555e-01,
           7.23318458e-02,  1.10511921e-01, -1.14771314e-01,
          -1.04336269e-01,  5.20934910e-02,  8.64798278e-02,
          -3.42181697e-02, -1.39057904e-01,  6.90201297e-02,
          -1.38434961e-01, -1.21509731e-01, -6.81641102e-02,
          -5.60541898e-02,  1.02614373e-01, -1.28788561e-01,
          -1.04955241e-01,  4.75839116e-02, -9.68884677e-02,
          -1.34384304e-01, -4.98847514e-02, -6.13117069e-02,
          -6.87620342e-02,  1.26426890e-01, -3.98295224e-02,
          -8.26288760e-02,  5.69848120e-02],
         [ 1.02902994e-01,  2.44277064e-02, -1.90152153e-02,
           8.77223462e-02,  3.53516056e-03, -3.97515530e-03,
          -4.20684442e-02, -8.81229937e-02, -9.76604000e-02,
          -1.10747457e-01, -9.26680788e-02, -3.67778689e-02,
          -5.00759855e-02, -3.80803011e-02, -2.37469058e-02,
           4.25817668e-02, -7.86229968e-